In [1]:
import find_eoms
import find_A_theta

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader

from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [10]:
import numpy as np

# Number of samples
q1_low  = -torch.pi
q1_high =  torch.pi
q2_low  = -torch.pi
q2_high =  torch.pi

n_samples = 100000

# Generate uniformly distributed points for q1 and q2
#q1 = torch.random.uniform(low=-10, high=10, size=n_samples)
#q2 = torch.random.uniform(low=-10, high=10, size=n_samples)
q1 = torch.linspace(q1_low, q1_high, n_samples)
q2 = torch.linspace(q2_low, q2_high, n_samples)

idx = torch.randperm(q2.shape[0])

q2 = q2[idx]

# Stack q1 and q2 to get the 2D coordinates
points = torch.stack([q1, q2], axis=1)  # Shape will be (1000, 2)

In [11]:
# Use TensorDataset to create the dataset
dataset = TensorDataset(points)

# Create the DataLoader with batch size and shuffling
batch_size = 128
dataloader = DataLoader(dataset, 
                        batch_size=batch_size, 
                        shuffle=True,
                        num_workers=0,
                        pin_memory=True)

print(dataloader)

# Example usage: iterate through the DataLoader
for batch in dataloader:
    #print(batch)
    break  # Just to show one batch

In [12]:
dataiter = iter(dataloader)
coordinates = next(dataiter)
print(coordinates)


[tensor([[-2.1154,  1.9703],
        [-1.3315, -2.3488],
        [ 2.5297,  0.6821],
        [-1.2382, -2.4014],
        [-2.0102, -3.0135],
        [-0.1951, -0.5361],
        [-2.0105,  2.5608],
        [-0.6164, -0.3898],
        [-1.2924,  3.0411],
        [-2.8575,  2.9053],
        [-0.3390, -1.0010],
        [-0.3670,  1.9386],
        [ 0.8663, -0.6847],
        [ 0.3119, -2.6506],
        [ 1.6587,  0.2681],
        [ 3.0300, -0.7150],
        [-1.5076,  0.3899],
        [ 2.9806,  1.9143],
        [-1.5551, -0.9392],
        [-1.5233, -1.5169],
        [ 0.0485, -0.9031],
        [-1.3892, -1.0113],
        [ 0.1057, -1.4576],
        [ 0.7592,  2.2269],
        [-0.2960, -2.5818],
        [-2.3324, -1.2142],
        [-1.8596, -1.4751],
        [ 2.7273, -1.3932],
        [-1.8457, -2.1110],
        [ 2.6372,  2.4313],
        [ 3.0292,  2.3202],
        [ 2.4058, -1.7689],
        [-2.1925,  1.7830],
        [ 2.7160,  2.8994],
        [ 1.0446,  0.3230],
        [-0.8281, -

In [13]:
class SinCosLayer(nn.Module):
    def __init__(self):
        super(SinCosLayer, self).__init__()

    def forward(self, x):
        # Apply sin() and cos() to both coordinates
        x_sin = torch.sin(x)
        x_cos = torch.cos(x)
        x_sin_cos_shape = (x.shape[0], x.shape[1]*2)
        x_sin_cos = torch.empty(x_sin_cos_shape, dtype=x_sin.dtype, device=x.device)
        x_sin_cos[:,0::2] = x_sin
        x_sin_cos[:,1::2] = x_cos
        return x_sin_cos
    
# Custom layer to reverse the interleaved sin() and cos() back to original coordinates
class InverseSinCosLayer(nn.Module):
    def __init__(self):
        super(InverseSinCosLayer, self).__init__()

    def forward(self, x):
        # x contains interleaved sin() and cos() values
        # Assuming input is of shape (batch_size, 4) for 2D coordinates
        sin_vals = x[:, 0::2]  # Extract sin values
        cos_vals = x[:, 1::2]  # Extract cos values

        # Use atan2 to recover the original angles from sin and cos
        original_coords = torch.atan2(sin_vals, cos_vals)
        return original_coords
    



class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            #SinCosLayer(),
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 2)#,
            #InverseSinCosLayer()
        )
        
        self.decoder = nn.Sequential(
            #SinCosLayer(),
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 2)#,
            #InverseSinCosLayer()
        )
    
    def forward(self, q):
        theta = self.encoder(q)
        q_hat = self.decoder(theta)
        return(theta, q_hat)

In [16]:
%%time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Autoencoder().to(device)  # Move model to GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
#scheduler.get_last_lr()


num_epochs = 50
outputs = []
for epoch in range(num_epochs):
    for (q) in dataloader:
        q = q[0].to(device)
        theta, q_hat = model(q)
        loss_reconstruction = criterion(q_hat, q)
        loss_theta = criterion(theta, q)
        loss = loss_reconstruction + loss_theta
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    #print(scheduler.get_last_lr())
        
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.7f}')
    outputs.append((epoch, q, q_hat, theta))

Epoch:1, Loss:0.0014420
Epoch:2, Loss:0.0006857
Epoch:3, Loss:0.0002579
Epoch:4, Loss:0.0002610
Epoch:5, Loss:0.0001433
Epoch:6, Loss:0.0001193
Epoch:7, Loss:0.0001049
Epoch:8, Loss:0.0000627
Epoch:9, Loss:0.0000378
Epoch:10, Loss:0.0000345
Epoch:11, Loss:0.0000698
Epoch:12, Loss:0.0000382
Epoch:13, Loss:0.0000402
Epoch:14, Loss:0.0000237
Epoch:15, Loss:0.0000133
Epoch:16, Loss:0.0000260
Epoch:17, Loss:0.0000180
Epoch:18, Loss:0.0000105
Epoch:19, Loss:0.0000123
Epoch:20, Loss:0.0000182
Epoch:21, Loss:0.0000101
Epoch:22, Loss:0.0000089
Epoch:23, Loss:0.0000083
Epoch:24, Loss:0.0000061
Epoch:25, Loss:0.0000086
Epoch:26, Loss:0.0000039
Epoch:27, Loss:0.0000120
Epoch:28, Loss:0.0000054
Epoch:29, Loss:0.0000069
Epoch:30, Loss:0.0000030
Epoch:31, Loss:0.0000052
Epoch:32, Loss:0.0000055
Epoch:33, Loss:0.0000039
Epoch:34, Loss:0.0000056
Epoch:35, Loss:0.0000096
Epoch:36, Loss:0.0000047
Epoch:37, Loss:0.0000083
Epoch:38, Loss:0.0000076
Epoch:39, Loss:0.0000066
Epoch:40, Loss:0.0000062
Epoch:41,

In [17]:
print("q\n", outputs[-1][1][0:4])
print("q_hat\n", outputs[-1][2][0:4])
print("theta\n", outputs[-1][3][0:4])
print("\n\n")
print("q_hat - q\n", outputs[-1][2][0:4] - outputs[-1][1][0:4])
print("theta - q\n", outputs[-1][3][0:4] - outputs[-1][1][0:4])

q
 tensor([[ 0.3717, -2.5508],
        [-3.0790, -1.4211],
        [-2.9599,  3.0781],
        [ 2.2007, -1.8303]])
q_hat
 tensor([[ 0.3728, -2.5512],
        [-3.0792, -1.4213],
        [-2.9585,  3.0779],
        [ 2.2005, -1.8312]], grad_fn=<SliceBackward0>)
theta
 tensor([[ 0.3728, -2.5513],
        [-3.0790, -1.4207],
        [-2.9591,  3.0786],
        [ 2.2005, -1.8305]], grad_fn=<SliceBackward0>)



q_hat - q
 tensor([[ 0.0011, -0.0004],
        [-0.0002, -0.0003],
        [ 0.0014, -0.0001],
        [-0.0003, -0.0008]], grad_fn=<SubBackward0>)
theta - q
 tensor([[ 1.1156e-03, -5.0879e-04],
        [ 4.5776e-05,  3.5501e-04],
        [ 8.1205e-04,  5.3859e-04],
        [-2.6774e-04, -1.7619e-04]], grad_fn=<SubBackward0>)


In [18]:
for param in model.parameters():
  print(param.data)

tensor([[-0.2104,  0.7211],
        [-0.7018, -0.2157],
        [ 0.1011,  0.1687],
        [-0.2055, -0.8071],
        [ 0.1568,  0.6023],
        [ 0.3777,  0.1138],
        [ 0.8109, -0.5030],
        [ 0.7592,  0.3492],
        [ 0.2322,  0.5386],
        [ 0.3779, -0.5448],
        [ 0.1411,  0.6850],
        [-0.5886, -0.2696],
        [-0.7537, -0.2392],
        [-0.2868,  0.4169],
        [-0.5807,  0.4712],
        [ 0.3062,  0.5910]])
tensor([ 0.0706, -0.2497,  0.8106,  0.6321, -0.4687,  0.1221,  0.5966, -0.5262,
         0.0573,  0.5863,  0.2258,  0.4140, -0.2025, -0.4419, -0.1210, -0.0748])
tensor([[-9.8366e-02,  1.6375e-01,  1.6134e-01,  ..., -4.3501e-02,
         -1.4464e-01,  9.0329e-02],
        [-4.8704e-03,  3.3929e-02, -1.7359e-01,  ..., -7.5324e-03,
         -1.8755e-01, -7.0220e-02],
        [ 4.7772e-02,  9.0448e-02,  2.4256e-01,  ..., -1.6549e-01,
          1.1638e-01,  1.8381e-01],
        ...,
        [ 4.2729e-02, -5.7124e-02, -1.4421e-02,  ..., -9.1929e-02,
 

In [19]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2104,  0.7211],
        [-0.7018, -0.2157],
        [ 0.1011,  0.1687],
        [-0.2055, -0.8071],
        [ 0.1568,  0.6023],
        [ 0.3777,  0.1138],
        [ 0.8109, -0.5030],
        [ 0.7592,  0.3492],
        [ 0.2322,  0.5386],
        [ 0.3779, -0.5448],
        [ 0.1411,  0.6850],
        [-0.5886, -0.2696],
        [-0.7537, -0.2392],
        [-0.2868,  0.4169],
        [-0.5807,  0.4712],
        [ 0.3062,  0.5910]], requires_grad=True)
Parameter containing:
tensor([ 0.0706, -0.2497,  0.8106,  0.6321, -0.4687,  0.1221,  0.5966, -0.5262,
         0.0573,  0.5863,  0.2258,  0.4140, -0.2025, -0.4419, -0.1210, -0.0748],
       requires_grad=True)
Parameter containing:
tensor([[-9.8366e-02,  1.6375e-01,  1.6134e-01,  ..., -4.3501e-02,
         -1.4464e-01,  9.0329e-02],
        [-4.8704e-03,  3.3929e-02, -1.7359e-01,  ..., -7.5324e-03,
         -1.8755e-01, -7.0220e-02],
        [ 4.7772e-02,  9.0448e-02,  2.4256e-01,  ..., -1.6549e-01,
    